In [1]:
from numpy.linalg import norm
import numpy as np
from scipy import sparse
from matplotlib import pyplot as plt
from scipy import optimize
from scipy.sparse import csr_matrix
from scipy.optimize import minimize
from sklearn.datasets import load_iris
from sklearn.datasets import load_svmlight_file
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.utils.validation import check_is_fitted, check_array, check_X_y, check_random_state
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import ParameterGrid,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report

from LMNN import LargeMarginNearestNeighbor
from LMNN_SS import SemiSupervisedLargeMarginNearestNeighbor

import time

# Metrics Learning 

**Group Members: Oskar Hint , Xiaoshen Hou,  Kasper Schønberg,  Valentine Van Der Nya**




***
## Tuning hyper-parameters

In [18]:
###Benchmark Grid Search Hyperparameters K, omega1,2, 3 #############
K=[1,2,3,4,5]
omega0= np.arange(0.1,1,0.1)
omega1= np.arange(0.1,1,0.1)
omega2= np.arange(0.1,1,0.1)

knn_param_grid = {'n_neighbors': K}
lmnn_param_grid = {'n_neighbors': K, 'omega': omega0}
ssc_param_grid = {'n_neighbors': K, 'omega0': omega0,'omega1': omega1,'omega2': omega2}

***
## 1.  Experinment 1 - USPS data 

<img src="ml_data/USPS_MNIST.gif">

In [3]:
print ("Parameters Grid for LMNN test ")
lmnn_param_grid

Parameters Grid for LMNN test 


{'n_neighbors': [1, 2, 3, 4, 5],
 'omega': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])}

In [8]:
def get_USPS_training_data():
    data = load_svmlight_file('ml_data/usps/usps')
    return data[0], data[1]

def get_USPS_testing_data():
    data = load_svmlight_file('ml_data/usps/usps.t')
    return data[0], data[1]

X_train, y_train = get_USPS_training_data()
X_test, y_test = get_USPS_testing_data()

In [9]:
######## Sample label, unlabel set  from training #########
X_train=np.asarray(X_train.todense())
y_train=np.asarray(y_train)
X_test=np.asarray(X_test.todense())
y_test=np.asarray(y_test)

sample_inds=np.random.randint(X_train.shape[0], size=20)
X_train_labelled=X_train[sample_inds, :]
y_train_labelled=y_train[sample_inds]

new_X_train = np.delete(X_train, sample_inds,0)
new_y_train = np.delete(y_train, sample_inds,0)

sample_inds=np.random.randint(new_X_train.shape[0], size=2000)
X_train_unlabelled=new_X_train[sample_inds, :]



In [25]:
##### KNN TUNNING USPS ########
knnclf = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=3, n_jobs=56,verbose=3)
knnclf.fit(X_train_labelled, y_train_labelled)

print("KNN Best parameters set found on development set:")
print()
print(knnclf.best_params_)
print()
print("Grid scores on development set:")
print()
means = knnclf.cv_results_['mean_test_score']
stds = knnclf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, knnclf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, knnclf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] n_neighbors=1 ...................................................
[CV] n_neighbors=2 ...................................................
[CV] n_neighbors=1 ...................................................
[CV] n_neighbors=2 ...................................................
[CV] n_neighbors=2 ...................................................
[CV] n_neighbors=3 ...................................................
[CV] n_neighbors=4 ...................................................
[CV] n_neighbors=1 ...................................................
[CV] n_neighbors=4 ...................................................
[CV] n_neighbors=4 ...................................................
[CV] n_neighbors=5 ...................................................
[CV] n_neighbors=3 ...................................................
[CV] ......................... n_neighbors=2, score=0.5, total=   0.0s
[CV] ............

[Parallel(n_jobs=56)]: Done   6 out of  15 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=56)]: Done  12 out of  15 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=56)]: Done  15 out of  15 | elapsed:    0.1s finished


KNN Best parameters set found on development set:
()
{'n_neighbors': 1}
()
Grid scores on development set:
()
0.650 (+/-0.379) for {'n_neighbors': 1}
0.450 (+/-0.100) for {'n_neighbors': 2}
0.500 (+/-0.265) for {'n_neighbors': 3}
0.450 (+/-0.100) for {'n_neighbors': 4}
0.450 (+/-0.321) for {'n_neighbors': 5}
()
Detailed classification report:
()
The model is trained on the full development set.
The scores are computed on the full evaluation set.
()
             precision    recall  f1-score   support

        1.0       0.76      0.73      0.74       359
        2.0       0.66      0.97      0.79       264
        3.0       0.79      0.44      0.56       198
        4.0       0.63      0.54      0.58       166
        5.0       0.00      0.00      0.00       200
        6.0       0.43      0.81      0.56       160
        7.0       0.49      0.85      0.62       170
        8.0       0.62      0.47      0.53       147
        9.0       0.00      0.00      0.00       166
       10.0     

In [20]:
##### LMNN TUNNING USPS ########  (Long execution!!)
lmnnclf = GridSearchCV(LargeMarginNearestNeighbor(), lmnn_param_grid, cv=3, n_jobs=56, verbose=3)
lmnnclf.fit(X_train_labelled, y_train_labelled)

print("LMNN Best parameters set found on development set:")
print()
print(lmnnclf.best_params_)
print()
print("Grid scores on development set:")
print()
means = lmnnclf.cv_results_['mean_test_score']
stds = lmnnclf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, lmnnclf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, lmnnclf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Fitting 3 folds for each of 45 candidates, totalling 135 fits
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.2 ........................................
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.2 ........................................
[CV] n_neighbors=1, omega=0.2 ........................................
[CV] n_neighbors=1, omega=0.30000000000000004 ........................
[CV] n_neighbors=1, omega=0.30000000000000004 ........................
[CV] n_neighbors=1, omega=0.30000000000000004 ........................
[CV] n_neighbors=1, omega=0.4 ........................................
[CV] n_neighbors=1, omega=0.4 ........................................
[CV] n_neighbors=1, omega=0.5 ........................................
[CV] n_neighbors=1, omega=0.4 ........................................
[CV] n_neighbor

[Parallel(n_jobs=56)]: Done  16 tasks      | elapsed:  1.9min


[CV]  n_neighbors=1, omega=0.7000000000000001, score=0.666666666667, total= 1.9min
[CV] n_neighbors=3, omega=0.7000000000000001 .........................
[CV] .............. n_neighbors=1, omega=0.4, score=0.5, total= 2.0min
[CV] n_neighbors=3, omega=0.7000000000000001 .........................
[CV] .............. n_neighbors=1, omega=0.9, score=0.5, total= 2.0min
[CV] n_neighbors=3, omega=0.7000000000000001 .........................
[CV] ... n_neighbors=2, omega=0.6, score=0.666666666667, total= 2.0min
[CV] n_neighbors=3, omega=0.8 ........................................
[CV] .............. n_neighbors=2, omega=0.4, score=0.3, total= 2.0min
[CV] n_neighbors=3, omega=0.8 ........................................
[CV] ... n_neighbors=2, omega=0.4, score=0.666666666667, total= 2.0min
[CV] n_neighbors=3, omega=0.8 ........................................
[CV] ............. n_neighbors=2, omega=0.8, score=0.25, total= 2.1min
[CV] n_neighbors=3, omega=0.9 ...................................

[Parallel(n_jobs=56)]: Done  70 out of 135 | elapsed:  4.1min remaining:  3.8min


[CV]  n_neighbors=4, omega=0.30000000000000004, score=0.6, total= 1.7min
[CV] n_neighbors=5, omega=0.7000000000000001 .........................
[CV]  n_neighbors=3, omega=0.7000000000000001, score=0.666666666667, total= 2.2min
[CV] n_neighbors=5, omega=0.7000000000000001 .........................
[CV] .............. n_neighbors=3, omega=0.1, score=0.5, total= 3.7min
[CV] n_neighbors=5, omega=0.7000000000000001 .........................
[CV] ... n_neighbors=4, omega=0.4, score=0.666666666667, total= 1.6min
[CV] n_neighbors=5, omega=0.8 ........................................
[CV] ... n_neighbors=3, omega=0.8, score=0.666666666667, total= 2.4min
[CV] n_neighbors=5, omega=0.8 ........................................
[CV] .............. n_neighbors=4, omega=0.2, score=0.5, total= 2.0min
[CV] n_neighbors=5, omega=0.8 ........................................
[CV] ... n_neighbors=3, omega=0.9, score=0.666666666667, total= 2.4min
[CV] n_neighbors=5, omega=0.9 .................................

[Parallel(n_jobs=56)]: Done 116 out of 135 | elapsed:  5.8min remaining:   57.3s


[CV] .............. n_neighbors=4, omega=0.5, score=0.5, total= 3.2min
[CV]  n_neighbors=5, omega=0.30000000000000004, score=0.666666666667, total= 2.3min
[CV] ... n_neighbors=5, omega=0.2, score=0.666666666667, total= 2.4min
[CV] .............. n_neighbors=5, omega=0.1, score=0.5, total= 2.7min
[CV]  n_neighbors=5, omega=0.7000000000000001, score=0.75, total= 1.8min
[CV] ... n_neighbors=4, omega=0.1, score=0.666666666667, total= 3.8min
[CV] ............. n_neighbors=2, omega=0.4, score=0.25, total= 6.1min
[CV] .............. n_neighbors=5, omega=0.9, score=0.5, total= 1.4min
[CV] ............. n_neighbors=5, omega=0.1, score=0.75, total= 2.7min
[CV] ... n_neighbors=5, omega=0.8, score=0.666666666667, total= 1.7min
[CV] ... n_neighbors=5, omega=0.9, score=0.666666666667, total= 1.7min
[CV] ............. n_neighbors=1, omega=0.4, score=0.75, total= 6.2min
[CV]  n_neighbors=5, omega=0.30000000000000004, score=0.75, total= 2.6min
[CV] .............. n_neighbors=5, omega=0.6, score=0.5, to

[Parallel(n_jobs=56)]: Done 135 out of 135 | elapsed:  7.3min finished


LMNN Best parameters set found on development set:
()
{'n_neighbors': 4, 'omega': 0.30000000000000004}
()
Grid scores on development set:
()
0.550 (+/-0.153) for {'n_neighbors': 1, 'omega': 0.1}
0.550 (+/-0.153) for {'n_neighbors': 1, 'omega': 0.2}
0.550 (+/-0.153) for {'n_neighbors': 1, 'omega': 0.30000000000000004}
0.600 (+/-0.208) for {'n_neighbors': 1, 'omega': 0.4}
0.550 (+/-0.153) for {'n_neighbors': 1, 'omega': 0.5}
0.550 (+/-0.153) for {'n_neighbors': 1, 'omega': 0.6}
0.550 (+/-0.153) for {'n_neighbors': 1, 'omega': 0.7000000000000001}
0.550 (+/-0.153) for {'n_neighbors': 1, 'omega': 0.8}
0.600 (+/-0.208) for {'n_neighbors': 1, 'omega': 0.9}
0.450 (+/-0.306) for {'n_neighbors': 2, 'omega': 0.1}
0.450 (+/-0.306) for {'n_neighbors': 2, 'omega': 0.2}
0.500 (+/-0.231) for {'n_neighbors': 2, 'omega': 0.30000000000000004}
0.400 (+/-0.351) for {'n_neighbors': 2, 'omega': 0.4}
0.400 (+/-0.351) for {'n_neighbors': 2, 'omega': 0.5}
0.300 (+/-0.493) for {'n_neighbors': 2, 'omega': 0.6}
0.

In [19]:
##### SSC LMNN TUNNING USPS ######## (Long execution!!)
sscclf = GridSearchCV(SemiSupervisedLargeMarginNearestNeighbor(), ssc_param_grid,fit_params={'X_unlabeled':X_train_unlabelled}, cv=3, n_jobs=56,verbose=3)
sscclf.fit(X_train_labelled, y_train_labelled)

print("SSC Best parameters set found on development set:")
print()
print(sscclf.best_params_)
print()
print("Grid scores on development set:")
print()
means = sscclf.cv_results_['mean_test_score']
stds = sscclf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, sscclf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, sscclf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Fitting 3 folds for each of 3645 candidates, totalling 10935 fits


/home/s132732/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_search.py:584: DeprecationWarning: "fit_params" as a constructor argument was deprecated in version 0.19 and will be removed in version 0.21. Pass fit parameters to the "fit" method instead.
  '"fit" method instead.', DeprecationWarning)


[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.1 ...............
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.1 ...............
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.1 ...............
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.2 ...............
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.2 ...............
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.2 ...............
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.30000000000000004 
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.30000000000000004 
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.30000000000000004 
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.4 ...............
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.4 ...............
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.5 ...............
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.4 ...............
[CV] n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.5 ...............
[CV

[Parallel(n_jobs=56)]: Done  16 tasks      | elapsed:  1.4min


[CV]  n_neighbors=1, omega1=0.2, omega0=0.1, omega2=0.2, score=1.0, total= 1.4min
[CV] n_neighbors=1, omega1=0.30000000000000004, omega0=0.1, omega2=0.7000000000000001 
[CV]  n_neighbors=1, omega1=0.2, omega0=0.1, omega2=0.2, score=0.666666666667, total= 1.5min
[CV] n_neighbors=1, omega1=0.30000000000000004, omega0=0.1, omega2=0.7000000000000001 
[CV]  n_neighbors=1, omega1=0.2, omega0=0.1, omega2=0.30000000000000004, score=1.0, total= 1.5min
[CV] n_neighbors=1, omega1=0.30000000000000004, omega0=0.1, omega2=0.7000000000000001 
[CV]  n_neighbors=1, omega1=0.2, omega0=0.1, omega2=0.8, score=1.0, total= 1.5min
[CV] n_neighbors=1, omega1=0.30000000000000004, omega0=0.1, omega2=0.8 
[CV]  n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.9, score=1.0, total= 1.6min
[CV] n_neighbors=1, omega1=0.30000000000000004, omega0=0.1, omega2=0.8 
[CV]  n_neighbors=1, omega1=0.1, omega0=0.1, omega2=0.6, score=0.5, total= 1.6min
[CV] n_neighbors=1, omega1=0.30000000000000004, omega0=0.1, omega2=0.8 
[CV]

...

[CV]  n_neighbors=5, omega1=0.9, omega0=0.9, omega2=0.8, score=0.75, total= 1.4min
[CV]  n_neighbors=5, omega1=0.9, omega0=0.9, omega2=0.9, score=0.2, total= 1.3min
[CV]  n_neighbors=5, omega1=0.7000000000000001, omega0=0.9, omega2=0.6, score=0.2, total= 3.9min
[CV]  n_neighbors=5, omega1=0.8, omega0=0.9, omega2=0.4, score=0.75, total= 2.9min
[CV]  n_neighbors=5, omega1=0.7000000000000001, omega0=0.9, omega2=0.4, score=0.5, total= 4.0min
[CV]  n_neighbors=5, omega1=0.9, omega0=0.9, omega2=0.1, score=0.3, total= 2.3min
[CV]  n_neighbors=5, omega1=0.9, omega0=0.9, omega2=0.5, score=0.2, total= 1.8min
[CV]  n_neighbors=5, omega1=0.8, omega0=0.9, omega2=0.30000000000000004, score=0.5, total= 3.2min
[CV]  n_neighbors=5, omega1=0.8, omega0=0.9, omega2=0.5, score=0.2, total= 3.0min
[CV]  n_neighbors=5, omega1=0.8, omega0=0.9, omega2=0.5, score=0.75, total= 2.9min
[CV]  n_neighbors=5, omega1=0.8, omega0=0.9, omega2=0.6, score=0.5, total= 2.7min
[CV]  n_neighbors=5, omega1=0.8, omega0=0.9, omeg

[Parallel(n_jobs=56)]: Done 10935 out of 10935 | elapsed: 507.2min finished


SSC Best parameters set found on development set:
()
{'n_neighbors': 1, 'omega1': 0.2, 'omega0': 0.4, 'omega2': 0.8}
()
Grid scores on development set:
()
0.700 (+/-0.306) for {'n_neighbors': 1, 'omega1': 0.1, 'omega0': 0.1, 'omega2': 0.1}
0.700 (+/-0.306) for {'n_neighbors': 1, 'omega1': 0.1, 'omega0': 0.1, 'omega2': 0.2}
0.650 (+/-0.379) for {'n_neighbors': 1, 'omega1': 0.1, 'omega0': 0.1, 'omega2': 0.30000000000000004}
0.650 (+/-0.379) for {'n_neighbors': 1, 'omega1': 0.1, 'omega0': 0.1, 'omega2': 0.4}
0.650 (+/-0.379) for {'n_neighbors': 1, 'omega1': 0.1, 'omega0': 0.1, 'omega2': 0.5}
0.650 (+/-0.379) for {'n_neighbors': 1, 'omega1': 0.1, 'omega0': 0.1, 'omega2': 0.6}
0.650 (+/-0.379) for {'n_neighbors': 1, 'omega1': 0.1, 'omega0': 0.1, 'omega2': 0.7000000000000001}
0.650 (+/-0.379) for {'n_neighbors': 1, 'omega1': 0.1, 'omega0': 0.1, 'omega2': 0.8}
0.650 (+/-0.379) for {'n_neighbors': 1, 'omega1': 0.1, 'omega0': 0.1, 'omega2': 0.9}
0.700 (+/-0.306) for {'n_neighbors': 1, 'omega1':

0.450 (+/-0.346) for {'n_neighbors': 4, 'omega1': 0.5, 'omega0': 0.5, 'omega2': 0.6}
0.450 (+/-0.346) for {'n_neighbors': 4, 'omega1': 0.5, 'omega0': 0.5, 'omega2': 0.7000000000000001}
0.500 (+/-0.404) for {'n_neighbors': 4, 'omega1': 0.5, 'omega0': 0.5, 'omega2': 0.8}
0.500 (+/-0.404) for {'n_neighbors': 4, 'omega1': 0.5, 'omega0': 0.5, 'omega2': 0.9}
0.500 (+/-0.265) for {'n_neighbors': 4, 'omega1': 0.6, 'omega0': 0.5, 'omega2': 0.1}
0.550 (+/-0.200) for {'n_neighbors': 4, 'omega1': 0.6, 'omega0': 0.5, 'omega2': 0.2}
0.500 (+/-0.265) for {'n_neighbors': 4, 'omega1': 0.6, 'omega0': 0.5, 'omega2': 0.30000000000000004}
0.450 (+/-0.346) for {'n_neighbors': 4, 'omega1': 0.6, 'omega0': 0.5, 'omega2': 0.4}
0.400 (+/-0.436) for {'n_neighbors': 4, 'omega1': 0.6, 'omega0': 0.5, 'omega2': 0.5}
0.450 (+/-0.346) for {'n_neighbors': 4, 'omega1': 0.6, 'omega0': 0.5, 'omega2': 0.6}
0.450 (+/-0.346) for {'n_neighbors': 4, 'omega1': 0.6, 'omega0': 0.5, 'omega2': 0.7000000000000001}
0.500 (+/-0.404) fo

             precision    recall  f1-score   support

        1.0       0.77      0.74      0.76       359
        2.0       0.66      0.97      0.79       264
        3.0       0.75      0.45      0.56       198
        4.0       0.62      0.50      0.55       166
        5.0       0.00      0.00      0.00       200
        6.0       0.40      0.81      0.54       160
        7.0       0.56      0.86      0.68       170
        8.0       0.57      0.49      0.53       147
        9.0       0.00      0.00      0.00       166
       10.0       0.35      0.63      0.45       177

avg / total       0.50      0.57      0.52      2007

()


In [26]:
print "KNN Best Hyperpara:", knnclf.best_params_
print "LMNN Best Hyperpara:", lmnnclf.best_params_
print "SSC Best Hyperpara:", sscclf.best_params_

KNN Best Hyperpara: {'n_neighbors': 1}
LMNN Best Hyperpara: {'n_neighbors': 4, 'omega': 0.30000000000000004}
SSC Best Hyperpara: {'n_neighbors': 1, 'omega1': 0.2, 'omega0': 0.4, 'omega2': 0.8}


In [21]:
print("######## Performance Test:  Labelled = ",X_train_labelled.shape, "  ###  Unlabelled = ", X_train_unlabelled.shape, "###########")
#k, omega0, omega1, omega2 =(4, 0.3,
#for k in K:
#    print("########  K = ",k, " ###########")
#    lmnn = LargeMarginNearestNeighbor(n_neighbors=k)
#    lmnn.fit(X_train_labelled,y_train_labelled)
#    print("USPS LMNN : ", lmnn.score(X_test,y_test))

#    ssc = SemiSupervisedLargeMarginNearestNeighbor(n_neighbors=k)
#    ssc.fit(X_train_labelled,X_train_unlabelled, y_train_labelled)
#    print("USPS SSC : ", ssc.score(X_test,y_test))
    
#    neigh = KNeighborsClassifier(n_neighbors=k)
#    neigh.fit(X_train_labelled,y_train_labelled)
#    print("USPS KNN : ",  neigh.score(X_test,y_test))

('######## Performance Test:  Labelled = ', (20, 256), '  ###  Unlabelled = ', (2000, 256), '###########')


***
## 2.  Experinment 2 - MNIST data 

In [9]:
mnist = fetch_mldata('MNIST original', data_home='ml_data/mnist')  #Download MMNIST

In [10]:
X, y = mnist.data, mnist.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

In [11]:
######## Sample label, unlabel set  from training #########
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)
X_test=np.asarray(X_test)
y_test=np.asarray(y_test)

sample_inds=np.random.randint(X_train.shape[0], size=20)
X_train_labelled=X_train[sample_inds, :]
y_train_labelled=y_train[sample_inds]

new_X_train = np.delete(X_train, sample_inds,0)
new_y_train = np.delete(y_train, sample_inds,0)

sample_inds=np.random.randint(new_X_train.shape[0], size=2000)
X_train_unlabelled=new_X_train[sample_inds, :]

In [13]:
##### LMNN TUNNING MNIST ########  (Long execution!!)
clf = GridSearchCV(LargeMarginNearestNeighbor(), lmnn_param_grid, cv=5, n_jobs=56, verbose=3)
clf.fit(X_train_labelled, y_train_labelled)

print("LMNN Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Fitting 10 folds for each of 45 candidates, totalling 450 fits
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.1 ........................................
[CV] n_neighbors=1, omega=0.2 ........................................
[CV] n_neighbors=1, omega=0.2 ........................................
[CV] n_neighbors=1, omega=0.2 ........................................
[CV] n_neighbo

[Parallel(n_jobs=56)]: Done  16 tasks      | elapsed: 189.4min


[CV] ............ n_neighbors=1, omega=0.2, score=0.75, total=191.2min
[CV] n_neighbors=1, omega=0.8 ........................................
[CV] .. n_neighbors=1, omega=0.4, score=0.818181818182, total=204.0min
[CV] n_neighbors=1, omega=0.8 ........................................
[CV]  n_neighbors=1, omega=0.30000000000000004, score=0.818181818182, total=210.1min
[CV] n_neighbors=1, omega=0.8 ........................................
[CV] .. n_neighbors=1, omega=0.2, score=0.818181818182, total=214.1min
[CV] n_neighbors=1, omega=0.8 ........................................
[CV] .. n_neighbors=1, omega=0.5, score=0.818181818182, total=221.3min
[CV] n_neighbors=1, omega=0.8 ........................................
[CV] .. n_neighbors=1, omega=0.6, score=0.818181818182, total=224.2min
[CV] n_neighbors=1, omega=0.8 ........................................
[CV] .. n_neighbors=1, omega=0.5, score=0.722222222222, total=226.7min
[CV] n_neighbors=1, omega=0.8 .................................

[CV] n_neighbors=2, omega=0.5 ........................................
[CV] n_neighbors=2, omega=0.5 ........................................
[CV] n_neighbors=2, omega=0.5 ........................................
[CV] n_neighbors=2, omega=0.6 ........................................
[CV] n_neighbors=2, omega=0.5 ........................................
[CV] n_neighbors=2, omega=0.5 ........................................
[CV] n_neighbors=2, omega=0.5 ........................................
[CV] n_neighbors=2, omega=0.5 ........................................
[CV] n_neighbors=2, omega=0.6 ........................................
[CV] n_neighbors=2, omega=0.6 ........................................
[CV] n_neighbors=2, omega=0.6 ........................................
[CV] n_neighbors=2, omega=0.6 ........................................
[CV] n_neighbors=2, omega=0.6 ........................................
[CV] n_neighbors=2, omega=0.6 ........................................
[CV] n

KeyboardInterrupt: 

In [ ]:
K=[1,2,3,4,5,6,7,8,9,10]
for k in K:
    print("########  K = ",k, " ###########")
    lmnn = LargeMarginNearestNeighbor(n_neighbors=k)
    lmnn.fit(X_train[0:200,],y_train[0:200])
    print("MNIST LMNN : ", lmnn.score(X_test,y_test))

    ssc = SemiSupervisedLargeMarginNearestNeighbor(n_neighbors=k)
    ssc.fit(X_train[0:200,],X_train[201:600,], y_train[0:200,])
    print("MNIST SSC : ", ssc.score(X_test.todense(),y_test))
    
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train[0:200,],y_train[0:200,])
    print("MNIST KNN : ", neigh.score(X_test,y_test))

## 3. Experiment - Sign Language Digits  https://www.kaggle.com/ardamavi/sign-language-digits-dataset/version/2

<img src="ml_data/SignLanguage.JPG">

In [ ]:
X = np.load('ml_data/Sign-language-digits-dataset/X.npy')
Y = np.load('ml_data/Sign-language-digits-dataset/Y.npy')
X = X.reshape((X.shape[0], X.shape[1]**2))
Y = np.argwhere(Y == 1)[:,1]

# sample into train and test datasets
train_test_sample_inds=np.random.randint(X.shape[0], size=100)

X_test = X[train_test_sample_inds]
y_test = Y[train_test_sample_inds]

X_train = np.delete(X, train_test_sample_inds,0)
y_train = np.delete(Y, train_test_sample_inds,0)


######## Sample label, unlabel set  from training #########
# keeping 1 to 20 ration of unlabeled to laneled data
sample_inds=np.random.randint(X_train.shape[0], size=200)
X_train_labelled=X_train[sample_inds,:]
y_train_labelled=y_train[sample_inds]

new_X_train = np.delete(X_train, sample_inds,0)
new_y_train = np.delete(y_train, sample_inds,0)

sample_inds=np.random.randint(new_X_train.shape[0], size=4000)
X_train_unlabelled=new_X_train[sample_inds,:]

In [ ]:
##### LMNN TUNNING SLD ########  (Long execution!!)
clf = GridSearchCV(LargeMarginNearestNeighbor(), lmnn_param_grid, cv=10, n_jobs=56)
clf.fit(X_train_labelled, y_train_labelled)

print("LMNN Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

In [ ]:
print ("Parameters Grid for SSC test ")
ssc_param_grid

In [ ]:
##### SSC LMNN TUNNING USPS ########
clf = GridSearchCV(SemiSupervisedLargeMarginNearestNeighbor(), ssc_param_grid,fit_params={'X_unlabeled':X_train_unlabelled}, cv=10, n_jobs=56)
clf.fit(X_train_labelled, y_train_labelled)

print("SCSC Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()